### Image To Image Translation Model

### Install pytorch_msssim

In [ ]:
!pip install pytorch_msssim

### Project path in Google Colab

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive", force_remount=True)

PROJECT_PATH = '/content/drive/My Drive/Research Hub/Projects/Present Projects/{Deep Hadad}/Implementation/DeepHadadProject/'

 ### Project path in Local Env

In [1]:
PROJECT_PATH = './'

In [ ]:
import sys
sys.path.append(PROJECT_PATH)

import dh_model_training as dh_train

### Constants

In [3]:
# Set paths
dh_train.PROJECT_PATH   = PROJECT_PATH

# Set paths
TRAINING_DATASET_PATH   = PROJECT_PATH + 'data/small_training_dataset'

### Custom Hyperparameters

In [4]:
batch_size = 38       # Batch size
num_epochs = 100      # Number of epochs

### Dataset Loading

In [ ]:
# Load the dataset
train_dataloader, val_dataloader = dh_train.load_dataset()

### Network initialisation

In [ ]:
# Instantiate the generator and discriminator
generator, discriminator = dh_train.instantiate_networks()

### Initialize optimizers

In [7]:
# Initialize optimizers
gen_optim, dis_optim = dh_train.init_optimizer(generator, discriminator)

### Model Training

In [ ]:
dh_train.network_training(train_dataloader, val_dataloader, generator, discriminator, gen_optim, dis_optim)